In [1]:
# Importar Paquetes
import numpy as np
import math
import time
import cv2
import imutils

numeros_ruleta = [0, 26, 3, 35, 12, 28, 7, 29, 18, 22, 9, 31, 14, 20, 1, 33, 16, 24, 5, 10, 23, 8, 30, 11, 36, 13, 27, 6, 34, 17, 25, 2, 21, 4, 19, 15, 32]

alfa = 0.026
r = 0.375

Beta = 0.075
velocidad_fallo = 3

rebote = 7
PinDom = 3

def Calcular_Numero(t0, tr0, tcorrido, Pin_Dom):
    
    ind = 0

    print(t0, tr0, tcorrido)
    
    b = math.sqrt(Beta/alfa)

    #print ("b: ", b)

    x = ((math.pow(np.e, alfa * 2 * 3.1415) - math.cosh(alfa * b * t0))/math.sinh(alfa * b * t0))

    #print ("x; ", x)

    c1 = math.pow(b,2)*(math.pow(x,2)-1)

    #print ("c1: ", c1)

    distancia_fallo = (1/(2 * alfa)) * math.log(c1 / (math.pow(velocidad_fallo,2) - math.pow(b,2)))

    c0 = -math.pow(np.arcsinh(b/math.sqrt(c1)),1)

    #print ("c0: ",c0)

    tiempo_fallo = (1/(alfa * b)) * (c0 - np.arcsinh(math.sinh(c0) * math.pow(np.e,(alfa * distancia_fallo))))

    print("tiempo fallo: ", tiempo_fallo)

    distancia_fallo = (1/(2 * alfa)) * math.log( np.sinh(c0 - (alfa * b * tiempo_fallo)) / np.sinh(c0))

    print ("distancia bola  fallo: ", distancia_fallo)

    circunferencia = 2 * 3.1415 * r 

    #print ("circunferencia: ", circunferencia)

    distancia_fallo_rps = (distancia_fallo / circunferencia) 

    #   print ("distancia bola fallo (rps): ", distancia_fallo_rps)

    distancia_rotor_fallo = (tiempo_fallo - t0 - tcorrido) / (tr0 * 1)
    
    print("vueltas rotor fallo: ", distancia_rotor_fallo)

    posicion_cero = 37 - (distancia_rotor_fallo - (int)(distancia_rotor_fallo))*37

    print("Posicion Cero: ", posicion_cero)

    #print("Pin Dom: ", Pin_Dom)

    if (Pin_Dom == 4):
        ind = (int)(posicion_cero)
        numero_golpea = numeros_ruleta[(int)(posicion_cero)] 

    if (Pin_Dom == 3):
        if ((int)(posicion_cero)+9 > 36):
            ind = (int)(posicion_cero)+9-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+9-36] 
        else:
            ind = (int)(posicion_cero)+9
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+9]
    
    if (Pin_Dom == 2):
        if ((int)(posicion_cero)+18 > 36):
            ind = (int)(posicion_cero)+18-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+18-36] 
        else:
            ind = (int)(posicion_cero)+18
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+18] 
    
    if (Pin_Dom == 1):
        if ((int)(posicion_cero)+27 > 36):
            ind = (int)(posicion_cero)+27-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+27-36] 
        else:
            ind = (int)(posicion_cero)+27
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+27] 

    print("Indice Golpea: ", ind)
    print("Numero Golpea: ", numero_golpea)

    if ((ind-rebote) <0):
        numero_ganador = numeros_ruleta[ind-rebote+36]
    else:
        numero_ganador = numeros_ruleta[ind-rebote]

    print("Numero Ganador: ", numero_ganador)

    
# Limites Inferior y Superior del Color del Pin
PinLower = (105, 105, 105)
PinUpper = (140, 140, 140)

# Limites Inferior y Superior del Color del Cero
CeroLower = (25, 45, 30)
CeroUpper = (35, 55, 40)

# Limites Inferior y Superior del Color de la Bola
BolaLower = (50, 50, 50)
BolaUpper = (55, 55, 55)

# Inicia Captura Video
cap = cv2.VideoCapture('C:/ruleta/Jafco.mp4')

print(cap.get(cv2.CAP_PROP_FPS))

time.sleep(1.0)

i = 0;

frames = 0;
frames_ant = 0;
vueltas = -1;
flag_vuelta = 0;
sumatoria = 0;
vueltas_rotor = -1;
frames_rotor_ant = 0;
frame_pos = 0;
tiempo_corrido = 0;
flag_calculo = 0;

#Salteo los primeros frames 
while i < 6400:
    
    # Lee Video por Frame 
    ret, frame = cap.read() 

    frames = frames + 1;

    i = i + 1;
    


while True:
    
    # Lee Video por Frame
                
    ret, frame = cap.read() 

    frames = frames + 1;
    
    # Si no pudo leer Frame sale
    if frame is None:
        break
    
    # Redimensiona el frame
    frame2 = imutils.resize(frame, width=800)

    
    # Crea un mascara con el rango de Color a Buscar
    maskpin = cv2.inRange(frame2, PinLower, PinUpper)
    maskcero = cv2.inRange(frame2, CeroLower, CeroUpper)
    maskbola = cv2.inRange(frame2, BolaLower, BolaUpper)
    
    # Encuentra el Contorno del Pin
    cntspin = cv2.findContours(maskpin.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    # Graba contorno ?
    cntspin = imutils.grab_contours(cntspin)
    if (len(cntspin)>0):

        #print("Frame: ", frames)

        #Dibujo de Marca para cumplir vuelta
        #cv2.circle(frame2, (300,67), 10, (0, 0, 255), 2)

        #Recorro Puntos del Contorno del Pin
        
        ymenor = 1000
        xmenor = 0

        for j, contour in enumerate(cntspin, start=1):

            #print(j)
            #print(contour)

            #print(contour.shape)
            
            sh = contour.shape[0]
            
            #print(sh)
        
            for k in range(sh-1):

                xpcontour = contour[k][0][0]
                ypcontour = contour[k][0][1] 
                
                if (ypcontour < ymenor):
                    ymenor = ypcontour
                    xmenor = xpcontour
            
        cv2.circle(frame2, (xmenor, ymenor+20), 1, (0, 0, 255), 10)

        cv2.circle(frame2, (xmenor, ymenor+80), 1, (0, 255, 255), 10)
        cv2.circle(frame2, (xmenor, ymenor+120), 1, (0, 255, 255), 10)

        #print(xmenor, ymenor)    
    
    # Encuentra el Contorno de la Bola
    cntsbola = cv2.findContours(maskbola.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Graba contorno ?
    cntsbola = imutils.grab_contours(cntsbola)
    if (len(cntsbola)>0):

       #print("Frame: ", frames)

        #Dibujo de Marca para cumplir vuelta
        cv2.circle(frame2, (300,67), 10, (0, 255, 255), 2)

        #Recorro Puntos del Contorno de la Bola
        for j, contour in enumerate(cntsbola, start=1):

            #print(j)
            #print(contour)

            #print(contour.shape)
            
            sh = contour.shape[0]
            
            #print(sh)
            
            pasobola = 0
            for k in range(sh-1):

                xcontour = contour[k][0][0]
                ycontour = contour[k][0][1] 
                #Dibujo cada Punto del Contorno de la Bola
                cv2.circle(frame2, (xcontour,ycontour), 1, (255, 255, 255), 2)

                #Si paso por marca es nueva vuelta
                if ((xcontour >= 295 and xcontour <= 305) 
                    and 
                    (ycontour >= 55 and ycontour <= 75)
                    and
                     (frames_ant + 10 < frames) 
                    ):
                    
                    #print("Si Bola")
                    pasobola = 1;
                    break;
    
            if (pasobola > 0):            
            
                vueltas = vueltas + 1

                if (vueltas == 0):   
                    
                    print("Vuelta 0")
                    
                    frames_ant = frames
                    
                    break;
            
                if (vueltas > 0):

                    #Posicion del Cero al cumplir Vuelta 1
                    if (vueltas == 1):
                    
                        flag_vuelta = 1;
                        
                        frame_pos = frames;
                        
                        print("Pos Rotor Vuelta 1")
                              
                        t0 = (frames-frames_ant)/25
                        
                        time.sleep(3.5)
     
                    sumatoria = sumatoria +  (frames-frames_ant)/25
                  
                    velocidad = 1/((frames-frames_ant)/25)
                    
                    print("Frame: ", frames, " Vuelta:", vueltas, "Tiempo x Vuelta ", (frames-frames_ant)/25, "Velocidad x Vuelta ", velocidad)
                    
                    frames_ant = frames;

                    break;

    # Encuentra el Contorno del Cero
    cntscero = cv2.findContours(maskcero.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Graba contorno ?
    cntscero = imutils.grab_contours(cntscero)
    if (len(cntscero)>0):
    
        #Dibujo de Marca para cumplir vuelta
        cv2.circle(frame2, (370,110), 10, (0, 255, 255), 2)
        cv2.circle(frame2, (370,150), 10, (0, 255, 255), 2)

        #Recorro Puntos del Contorno del Cero
        for l, contour in enumerate(cntscero, start=1):

            sh = contour.shape[0]
            
            pasocero = 0
            for k in range(sh-1):

                xrcontour = contour[k][0][0]
                yrcontour = contour[k][0][1]
                
                #Dibujo cada Punto del Contorno de la Bola
                cv2.circle(frame2, (xrcontour,yrcontour), 1, (255, 255, 0), 2)

                #Si paso por marca es nueva vuelta
                if ((xrcontour >= 320 and xrcontour <= 420)
                    and 
                    (yrcontour >= 90 and yrcontour <= 170)
                    and
                    frames_rotor_ant + 10 < frames
                    and
                    flag_vuelta == 1 
                    ):
                    
                    print("Si Cero")
                    pasocero = 1;
                    break;
            
            if (pasocero > 0):   
                
                vueltas_rotor = vueltas_rotor + 1
          
                tiempo_corrido = (frames - frame_pos)/25   
                
                if (vueltas_rotor == 0):   
                    
                    print("Vuelta Rotor 0")

                    tiempo_corrido = (frames - frame_pos)/25    
           
                    print("Frame: ", frames, "Frame Pos: ", frame_pos, "Tiempo Corrido: ", tiempo_corrido)
                    
                    frames_rotor_ant = frames
                    
                    break;
            
                if (vueltas_rotor > 0):
    
                    #print(xcontour, ycontour);
                    
                    tr0 = ((frames-frames_rotor_ant)/25)
                    
                    velocidad = 1/((frames-frames_rotor_ant)/25)
                    
                    print("Frame: ", frames, "Frame Rotor Ant", frames_rotor_ant, " Vuelta Rotor: ", vueltas_rotor, "Tiempo Rotor x Vuelta ", (frames-frames_rotor_ant)/25, "Velocidad Rotor x Vuelta", velocidad)
                    

                    frames_rotor_ant = frames;
                    
                    break;    
                    
                
    # Muestra el frame
    cv2.imshow("Framer", frame2)
    
    if (vueltas >= 1 and vueltas_rotor >=1 and flag_calculo == 0):
        Calcular_Numero(t0, tr0, tiempo_corrido, PinDom)
        flag_calculo = 1;

    # Captura tecla presionada
    key = cv2.waitKey(1) & 0xFF

    # Presione 'a' para Iniciar Tiempo
    if key == ord("a"):
        framea = frames
        
    # Presione 'b' para Terminar Tiempo
    if key == ord("s"):
        print("Tiempo Cron: ", (frames-framea)/25)

    # Presione 't' para Medir Tiempo
    if key == ord("t"):
        print("Tiempo Parcial: ", sumatoria)
        sumatoria = sumatoria +  (frames-frames_ant)/25
        print("Frame: ", frames, " Vuelta : ", "Final", "Tiempo x Vuelta ", (frames-frames_ant)/25)
        print("Tiempo Total: ", sumatoria)
    
    # Presione 'r' para Resetear
    if key == ord("r"):
        print("Reset !")
        frames = 0;
        frames_ant = 0;
        vueltas = -1;
        flag_vuelta = 0;
        sumatoria = 0;
        vueltas_rotor = -1
        frames_rotor_ant = 0
        frame_pos = 0;
        tiempo_corrido = 0;
        flag_calculo = 0;

    # Presione 'o' para Salir
    if key == ord("o"):
        time.sleep(30.5)
        
    # Presione 'q' para Salir
    if key == ord("q"):
        #print("Vueltas: ", vueltas)
        break

    time.sleep(0.08)

# Libera 
cap.release()

# Destruye Ventanas
cv2.destroyAllWindows()



25.000732229224916


AttributeError: module 'imutils' has no attribute 'grab_contours'